In [ ]:
import cv2
import dlib
import time
import numpy as np
from collections import Counter

# Load face detector and landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Indices for eye landmarks (Dlib 68-point model)
LEFT_EYE_INDICES = list(range(42, 48))
RIGHT_EYE_INDICES = list(range(36, 42))

def get_eye_aspect_ratio(eye_points):
    A = np.linalg.norm(eye_points[1] - eye_points[5])
    B = np.linalg.norm(eye_points[2] - eye_points[4])
    C = np.linalg.norm(eye_points[0] - eye_points[3])
    return (A + B) / (2.0 * C)

def analyze_eye_tracking(frame, landmarks):
    left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in LEFT_EYE_INDICES])
    right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in RIGHT_EYE_INDICES])
    
    left_ear = get_eye_aspect_ratio(left_eye)
    right_ear = get_eye_aspect_ratio(right_eye)
    
    avg_ear = (left_ear + right_ear) / 2.0
    
    if avg_ear < 0.25:
        return "Drowsy or Low Engagement"
    elif avg_ear < 0.3:
        return "Neutral"
    else:
        return "Highly Engaged"

def get_most_frequent_engagement(engagement_list):
    if not engagement_list:
        return "No engagement data collected"
    return Counter(engagement_list).most_common(1)[0][0]

def main():
    cap = cv2.VideoCapture(0)
    print("Starting AI-driven eye-tracking system...")
    
    engagement_records = []
    start_time = time.time()
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        
        for face in faces:
            landmarks = predictor(gray, face)
            engagement = analyze_eye_tracking(frame, landmarks)
            engagement_records.append(engagement)
            print(f"Detected Engagement: {engagement}")
        
        time.sleep(2)  # Simulate processing delay
        
        if time.time() - start_time >= 60:
            most_frequent_engagement = get_most_frequent_engagement(engagement_records)
            print(f"Most Frequent Engagement in Last Minute: {most_frequent_engagement}")
            engagement_records.clear()
            start_time = time.time()
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Starting AI-driven eye-tracking system...
Detected Engagement: Highly Engaged
Detected Engagement: Highly Engaged
Detected Engagement: Highly Engaged
Detected Engagement: Highly Engaged
Detected Engagement: Highly Engaged
Detected Engagement: Neutral
Detected Engagement: Highly Engaged
Detected Engagement: Highly Engaged
Detected Engagement: Neutral
Detected Engagement: Highly Engaged
Detected Engagement: Neutral
Detected Engagement: Neutral
Detected Engagement: Drowsy or Low Engagement
Detected Engagement: Highly Engaged
Detected Engagement: Drowsy or Low Engagement
Detected Engagement: Highly Engaged
Detected Engagement: Neutral
Detected Engagement: Drowsy or Low Engagement
Detected Engagement: Neutral
Most Frequent Engagement in Last Minute: Highly Engaged
Detected Engagement: Drowsy or Low Engagement
Detected Engagement: Drowsy or Low Engagement
Detected Engagement: Drowsy or Low Engagement
Detected Engagement: Neutral
Detected Engagement: Neutral
Detected Engagement: Neutral
Detec